In [3]:
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import wordnet

import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score

import textattack
from textattack.transformations.word_swaps import WordSwap
from textattack import AttackArgs
from textattack.datasets import Dataset
from textattack import Attacker
from textattack import Attack
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.attack_recipes import AttackRecipe
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper

In [4]:
# Fungsi untuk import dataset
def load_data(dataset_split="train"):
    df = pd.read_csv(f'LabelEncoded_{dataset_split}_data.csv')
    # Remove non-alphanumeric characters
    df["text"] = df["text"].apply(lambda x: re.sub("[^a-zA-Z]", " ", str(x)))
    df['text'] = df['text'].apply(lambda x: re.sub('  +', ' ', str(x)))
    df['text'] = df['text'].apply(lambda x: re.sub('\n', ' ', str(x)))
    df['text'] = df['text'].apply(lambda x: re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', str(x)))
    df["text"] = df["text"].apply(lambda x: re.sub("[^a-zA-Z]", " ", str(x)))
    # Tokenize the training and testing data
    df_tokenized = tokenize_review(df)
    return df_tokenized

# Fungsi untuk melakukan tokenizing dataset
def tokenize_review(df):
    # Tokenize Reviews in the DataFrame
    tokened_reviews = [word_tokenize(rev) for rev in df["text"]]
    
    # Initialize the Indonesian stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Stem tokens for each review in Indonesian
    stemmed_tokens = []
    for tokens in tokened_reviews:
        # Join the tokens into a single string for stemming, since Sastrawi expects a sentence
        sentence = " ".join(tokens)
        # Perform stemming on the whole sentence
        stemmed_sentence = stemmer.stem(sentence)
        stemmed_tokens.append(stemmed_sentence)

    # Insert the stemmed reviews into the DataFrame
    df.insert(1, column="Stemmed", value=stemmed_tokens)
    
    return df

In [5]:
# Import data train test validation
df_train = load_data("train")
df_test = load_data('test')
df_val = load_data('valid')

In [6]:
# Melakukan stemming pada dataset
X_train = df_train['Stemmed'].astype(str)
X_test = df_test['Stemmed'].astype(str)
X_val = df_val['Stemmed'].astype(str)
y_train = df_train['label']
y_test = df_test['label']
y_val = df_val['label']

In [7]:
# Melakukan label encoding pada label data
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
y_val = le.fit_transform(y_val)

In [8]:
# Inisialisasi tokenizer dari BERT
tokenizer = BertTokenizer.from_pretrained(
    'indolem/indobertweet-base-uncased',
    do_lower_case=True
)

In [9]:
# Encode the training data
encoded_data_train = tokenizer.batch_encode_plus(
    X_train,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

# Encode the validation data
encoded_data_val = tokenizer.batch_encode_plus(
    X_val,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

# # Encode the test data
encoded_data_test = tokenizer.batch_encode_plus(
    X_test,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

# Extract input IDs and attention masks for training data
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train)

# Extract input IDs and attention masks for validation data
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(y_val)

# Extract input IDs and attention masks for test data
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(y_test)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [10]:
# Menginisialisasi Tensor Dataset
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val, 
                            attention_masks_val,
                           labels_val)

dataset_test = TensorDataset(input_ids_test, 
                            attention_masks_test,
                           labels_test)

In [11]:
# Inisialisasi model dengan output 3 label
model = BertForSequenceClassification.from_pretrained(
                                      'indolem/indobertweet-base-uncased', 
                                      num_labels = 3,
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobertweet-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Inisialisasi ukuran batch saat training
batch_size = 4

# Inisialisasi dataloader dengan inputan Tensor Dataset, sampler, dan ukuran batch
dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

dataloader_test = DataLoader(
    dataset_test,
    sampler=RandomSampler(dataset_test),
    batch_size=1
)

In [13]:
# Inisialisasi Optimizer
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

c:\Python310\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
# Inisialisasi Epoch dan Scheduler
epochs = 3

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

In [15]:
# Fungsi untuk menampilkan f1_score saat training
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [16]:
# Fungsi untuk melihat seberapa akurasi perkelas dari model
def accuracy_per_class(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        print(f'Class: {label}')
        print(f'Accuracy: {len(y_preds[y_preds == label])}/{len(y_true)}\n')

In [17]:
# Pengecekan apakah device yang digunakan saat training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [18]:
# Fungsi untuk mengevaluasi saat training
def evaluate(dataloader_test):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_test):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_test) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [19]:
#Melakukan training sebanyak Epoch yang telah diinisialisasikan
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
    
    #torch.save(model.state_dict(), f'Models/BERT_ft_Epoch{epoch}.model')
    
    # Evaluasi per epoch
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_test)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/2750 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.34241611326786436


  0%|          | 0/500 [00:00<?, ?it/s]

Validation loss: 0.3873551219656365
F1 Score (weighted): 0.9120095821541961


...

Validation loss: 0.44956752958626023
F1 Score (weighted): 0.9162665938765298


Epoch 3:   0%|          | 0/2750 [00:00<?, ?it/s]


Epoch 3
Training loss: 0.12690628786471841


  0%|          | 0/500 [00:00<?, ?it/s]

Validation loss: 0.45388479330114206
F1 Score (weighted): 0.9123598388416498


In [20]:
accuracy_per_class(predictions, true_vals)

Class: 0
Accuracy: 204/204

Class: 1
Accuracy: 60/88

Class: 2
Accuracy: 194/208



In [21]:
original_model = model
original_tokenizer = tokenizer
# Wrapping model
model_wrapper = HuggingFaceModelWrapper(original_model, original_tokenizer)

In [22]:
# Menyimpan model
torch.save(original_model, 'pre-trained.pth')

In [23]:
# Membuat kelas untuk wordswap berbahasa Indonesia
class WordSwapWordNetBahasa(WordSwap):
    """Transforms an input by replacing its words with synonyms provided by
    WordNet.

    >>> from textattack.transformations import WordSwapWordNet
    >>> from textattack.augmentation import Augmenter

    >>> transformation = WordSwapWordNet()
    >>> augmenter = Augmenter(transformation=transformation)
    >>> s = 'I am fabulous.'
    >>> augmenter.augment(s)
    """
    # Melakukan setting bahasa saat kelas dipanggil
    def __init__(self, language="ind"):
        nltk.download("omw-1.4")
        # if language not in wordnet.langs():
        #     raise ValueError(f"Language {language} not one of {wordnet.langs()}")
        self.language = language

    # Mendapatkan sinonim untuk wordswap
    def _get_replacement_words(self, word, random=False):
        """Returns a list containing all possible words with 1 character
        replaced by a homoglyph."""
        synonyms = set()
        for syn in wordnet.synsets(word, lang=self.language):
            for syn_word in syn.lemma_names(lang=self.language):
                if (
                    (syn_word != word)
                    and ("_" not in syn_word)
                    and (textattack.shared.utils.is_one_word(syn_word))
                ):
                    # WordNet can suggest phrases that are joined by '_' but we ignore phrases.
                    synonyms.add(syn_word)
        return list(synonyms)


In [24]:
# Inisialisasi kelas untuk algoritma pencarian, tujuan, larangan, dan transformasi
class PWWSham2024INDO(AttackRecipe):
    
    @staticmethod
    def build(model_wrapper):
        transformation = WordSwapWordNetBahasa(language="ind")
        constraint = [RepeatModification(), StopwordModification(language='indonesian')]
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR("weighted-saliency")
        return Attack(goal_function, constraint, transformation, search_method)

In [25]:
custom_data = list(df_test[['text', 'label']].itertuples(index=False, name=None))

In [26]:
import logging
from textattack.datasets import Dataset

logging.basicConfig(filename="textattack_output.log", level=logging.INFO)

# Import dataset yang akan diserang
dataset = Dataset(custom_data) #HuggingFaceDataset("tyqiangz/multilingual-sentiments", "indonesian", "test")
# Inisilisasi serangan
attack = PWWSham2024INDO.build(model_wrapper=model_wrapper)
# Inisialisasi Detail serangan
attackargs = AttackArgs(num_examples=500, log_to_csv="attack_results_pretrained.csv")
attacker = Attacker(attack, dataset, attack_args=attackargs)
# Penyerangan Dataset
attack_results = attacker.attack_dataset()

for result in attack_results:
    logging.info(result)

[nltk_data] Downloading package omw-1.4 to C:\Users\Ilham
[nltk_data]     AM\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_pretrained.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNetBahasa
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/500 [00:50<7:03:00, 50.86s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[0 (100%)]] --> [[2 (98%)]]

kemarin gue datang ke tempat makan baru yang ada di dago atas gue kira makanan nya enak karena harga nya [[mahal]] ternyata boro boro tidak mau lagi deh ke tempat itu sudah mana tempat nya juga tidak [[nyaman]] banget terlalu [[sempit]] 

kemarin gue datang ke tempat makan baru yang ada di dago atas gue kira makanan nya enak karena harga nya [[mewah]] ternyata boro boro tidak mau lagi deh ke tempat itu sudah mana tempat nya juga tidak [[romantik]] banget terlalu [[kencang]] 




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   0%|          | 2/500 [01:16<5:19:30, 38.50s/it]

...

--------------------------------------------- Result 498 ---------------------------------------------
[[1 (67%)]] --> [[2 (100%)]]

salah satu kegemaran anak remaja indonesia sekarang adalah [[mendengar]] kpop 

salah satu kegemaran anak remaja indonesia sekarang adalah [[sabar]] kpop 




[Succeeded / Failed / Skipped / Total] 367 / 101 / 31 / 499: 100%|█████████▉| 499/500 [2:59:10<00:21, 21.54s/it]

--------------------------------------------- Result 499 ---------------------------------------------
[[2 (100%)]] --> [[0 (93%)]]

melihat warna [[hijau]] bisa bikin [[mata]] jadi lebih santai 

melihat warna [[mentah]] bisa bikin [[angka]] jadi lebih santai 




[Succeeded / Failed / Skipped / Total] 368 / 101 / 31 / 500: 100%|██████████| 500/500 [2:59:17<00:00, 21.51s/it]

--------------------------------------------- Result 500 ---------------------------------------------
[[1 (86%)]] --> [[0 (99%)]]

bondan winarno yang suka bilang maknyus sekarang sudah [[tiada]] 

bondan winarno yang suka bilang maknyus sekarang sudah [[tidak]] 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 368    |
| Number of failed attacks:     | 101    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 93.8%  |
| Accuracy under attack:        | 20.2%  |
| Attack success rate:          | 78.46% |
| Average perturbed word %:     | 13.38% |
| Average num. words per input: | 21.57  |
| Avg num queries:              | 325.67 |
+-------------------------------+--------+


In [27]:
from textattack.loggers import CSVLogger
from textattack.attack_results import SuccessfulAttackResult

pd.options.display.max_colwidth = (
    480  # increase colum width so we can actually read the examples
)

logger = CSVLogger(color_method="html")

for result in attack_results:
    if isinstance(result, SuccessfulAttackResult):
        logger.log_attack_result(result)

from IPython.core.display import display, HTML

results = pd.DataFrame.from_records(logger.row_list)
display(HTML(results[["original_text", "perturbed_text"]].to_html(escape=False)))

textattack: Logging to CSV at path results.csv
C:\Users\Ilham AM\AppData\Local\Temp\ipykernel_33068\2580654721.py:14: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


,original_text,perturbed_text
0,kemarin gue datang ke tempat makan baru yang ada di dago atas gue kira makanan nya enak karena harga nya mahal ternyata boro boro tidak mau lagi deh ke tempat itu sudah mana tempat nya juga tidak nyaman banget terlalu sempit,kemarin gue datang ke tempat makan baru yang ada di dago atas gue kira makanan nya enak karena harga nya mewah ternyata boro boro tidak mau lagi deh ke tempat itu sudah mana tempat nya juga tidak romantik banget terlalu kencang
1,kayak nya sih gue tidak akan mau balik lagi ke tempat itu gila ya gue enggak ngerti kenapa tempat nya dibiarkan panas sudah begitu kotor pula kalau panas kepanasan kalau hujan kehujanan harus nya sih tidak ada restoran yang kayak gitu tidak tahu deh apa yang mereka jual,menyenangi nya sih gue tidak akan mau balik lagi ke tempat itu kegemaran ya gue enggak ngerti kenapa tempat nya dibiarkan panas sudah begitu menyeramkan pula kalau panas kepanasan kalau hujan kehujanan harus nya sih tidak ada restoran yang kayak gitu tidak tahu deh apa yang mereka jual
2,ini pertama kalinya gua ke bank buat ngurusin pembuatan rekening baru nama nya juga orang pertama kali ya baru ke bank gua kena semprot kelihatan banget pelayanan pelanggan nya tidak suka gua banyak bertanya amit amit padahal itu kan tugas mereka buat melayangkan gua,ini pertama kalinya gua ke bank buat ngurusin pembuatan rekening baru nama nya juga orang pertama kali ya baru ke bank gua berhasil semprot kelihatan banget pelayanan pelanggan nya tidak menyenangi gua banyak bertanya amit amit padahal itu kan tugas mereka buat melayangkan gua
3,pelayanan di hotel salak bogor tidak sebagus yang gue membayangkan fasilitas nya juga biasa banget padahal kata nya hotel bintang lima hm kecewa kayak nya sih nanti nanti tidak mau ke sana lagi,pelayanan di hotel salak bogor tidak sebagus yang gue membayangkan fasilitas nya juga biasa banget padahal kata nya hotel bintang lima hm menjelajahi suka nya sih nanti nanti tidak mau ke sana lagi
4,ada apa dengan young lex kenapa dia harus mengirim hal hal tidak berfaedah kayak gitu merusak moral banget sumpah,ada apa dengan young lex kenapa dia harus mengirim hal hal tidak berfaedah menggemari gitu menggeledah semangat banget sumpah
5,saya itu memesan tiket dot com sudah dibayar email belum ada balasan bukti pembayaran sudah saya kirim by email saya masukkan data balasan nya pemesanan kadaluarsa terus akhirnya saya beli lagi langsung ke loket saya minta uang kembali masih menunggu hari ambil saja uang tiket nya tidak apa apa sedih saya download aplikasi ini,saya itu memesan tiket dot com sudah dibayar email belum ada balasan bukti pembayaran sudah saya kirim by email saya masukkan data balasan nya pemesanan kadaluarsa terus akhirnya saya beli lagi langsung ke loket saya minta uang kembali masih menunggu hari ambil saja uang tiket nya tidak apa apa terharu saya download aplikasi ini
6,saya sudah bayar tetapi etiket tidak dikirim malah kadaluwarsa diminta struk pembayaran saya sudah tidak punya hanya sms banking tetapi tetap saja tidak digubris dari pelayanan pelanggan nya sudah email dan telepon berkali berkali tetap tidak ada hasil sangat mengecewakan,saya sudah bayar tetapi etiket tidak dikirim malah kadaluwarsa diminta struk penyelesaian saya sudah tidak punya hanya sms banking tetapi tetap saja tidak digubris dari layanan pelanggan nya sudah email dan telepon berkali berkali tetap tidak ada pulangan sangat menggiurkan
7,saya mesan tiket tapi tidak dikonformasi dan proses pengembalian uang yang tidak jelas dan belum ada sampai sekarang pelayanan nya yang sangat buruk dan saran saya jangan pesan di sini sangat mengecewakan,saya mesan tiket tapi tidak dikonformasi dan proses pengembalian uang yang tidak jelas dan belum ada sampai sekarang pelayanan nya yang sangat murah dan saran saya jangan pesan di sini sangat mengentuti
8,sangat kecewa dengan aplikasi ini sudah ditransfer tetapi etiket tidak dikirim saya coba telepon call centre seharian tetapi sibuk terus call genre